<a href="https://colab.research.google.com/github/emrehangorgec/GANsNFTs/blob/main/GANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN

In [ ]:
!git clone https://github.com/NVlabs/stylegan3.git

Cloning into 'stylegan3'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 212 (delta 0), reused 1 (delta 0), pack-reused 207
Receiving objects: 100% (212/212), 4.17 MiB | 9.85 MiB/s, done.
Resolving deltas: 100% (98/98), done.


In [ ]:
%cd stylegan3

/content/stylegan3


In [ ]:
!wget https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-t-ffhq-1024x1024.pkl


--2024-04-04 19:51:52--  https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-t-ffhq-1024x1024.pkl
Resolving api.ngc.nvidia.com (api.ngc.nvidia.com)... 34.212.239.79, 35.167.170.225
Connecting to api.ngc.nvidia.com (api.ngc.nvidia.com)|34.212.239.79|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://files.ngc.nvidia.com/org/nvidia/team/research/models/stylegan3/versions/1/files/stylegan3-t-ffhq-1024x1024.pkl?Expires=1712263913&Signature=zVziK0vRWpXI6PmPoN41avSMA7kL0UzN8cy2NRuF8Rs7AsFjfXdCZjn78AOSNvEtFUwNuBW9uKtLcssHiNXFPuZ67FPV-EixtmsDoP6mR~MQy10MXdsH52i1Iw9KrwiQcZ96zULqDRXRnetOSwTIsfG2MMdNLLl1B38OhReZOdXs5ipIfLtCTxmsAf-9bAYSE1yVsU5Lua9wiyTw1B5LHvyPczgmpM3WaOgyi7wrW7I6v4bS0V2rU6UheGDMg1-xLZo4LUXqE5Erd-QjEXLxmRbD6b2MJEBvx1W1oKJZHWE3942DP-FQ7OSSOuhNhiZX9cpALGxhTgwziPuCuoNB4w__&Key-Pair-Id=KCX06E8E9L60W [following]
--2024-04-04 19:51:53--  https://files.ngc.nvidia.com/org/nvidia/team/research/models/stylegan3/version

In [ ]:
!pip install ninja

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 5.4 MB/s eta 0:00:00


In [ ]:
!ls /content/stylegan3

avg_spectra.py	 docs		  legacy.py    stylegan3-t-ffhq-1024x1024.pkl  visualizer.py
calc_metrics.py  environment.yml  LICENSE.txt  stylegan3-t-ffhqu-256x256.pkl   viz
dataset_tool.py  gen_images.py	  metrics      torch_utils
dnnlib		 gen_video.py	  __pycache__  training
Dockerfile	 gui_utils	  README.md    train.py


In [ ]:
# HIDE OUTPUT
URL = "https://api.ngc.nvidia.com/v2/models/nvidia/research/"\
      "stylegan3/versions/1/files/stylegan3-r-ffhq-1024x1024.pkl"
_URL = "https://api.ngc.nvidia.com/v2/models/org/nvidia/team/research/stylegan3/1/files?redirect=true&path=stylegan3-r-afhqv2-512x512.pkl"

!python /content/stylegan3/gen_images.py \
    --network={_URL} \
  --outdir=/content/results --seeds=6600-6625

Loading networks from "https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhq-1024x1024.pkl"...
Generating image for seed 6600 (0/26) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.
Generating image for seed 6601 (1/26) ...
Generating image for seed 6602 (2/26) ...
Generating image for seed 6603 (3/26) ...
Generating image for seed 6604 (4/26) ...
Generating image for seed 6605 (5/26) ...
Generating image for seed 6606 (6/26) ...
Generating image for seed 6607 (7/26) ...
Generating image for seed 6608 (8/26) ...
Generating image for seed 6609 (9/26) ...
Generating image for seed 6610 (10/26) ...
Generating image for seed 6611 (11/26) ...
Generating image for seed 6612 (12/26) ...
Generating image for seed 6613 (13/26) ...
Generating image for seed 6614 (14/26) ...
Generating image for seed 6615 (15/26) ...
Generating image for seed 6616 (16/26) ...
Generating image for seed 661

In [ ]:
# HIDE OUTPUT
URL = "https://api.ngc.nvidia.com/v2/models/nvidia/research/"\
      "stylegan3/versions/1/files/stylegan3-r-ffhq-1024x1024.pkl"
_URL = "https://api.ngc.nvidia.com/v2/models/nvidia/research/"\
      "stylegan3/versions/1/files/stylegan3-r-afhqv2-512x512.pkl"

!python /content/stylegan3/gen_images.py \
    --network={_URL} \
  --outdir=/content/results --seeds=6626-6630

Loading networks from "https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-afhqv2-512x512.pkl"...
Generating image for seed 6626 (0/5) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.
Generating image for seed 6627 (1/5) ...
Generating image for seed 6628 (2/5) ...
Generating image for seed 6629 (3/5) ...
Generating image for seed 6630 (4/5) ...


## **Trainign our own dataset**

In [ ]:
!nvidia-smi

Tue Apr 16 14:02:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
CMD = "python /content/stylegan2-ada-pytorch/dataset_tool.py "\
  "--source /content/drive/MyDrive/GANsNFTs/StyleGAN/data/gan/images "\
  "--dest /content/drive/MyDrive/GANsNFTs/StyleGAN/data/gan/dataset/portraits"

!{CMD}

100% 560/560 [00:14<00:00, 38.70it/s]


In [ ]:
!pip uninstall torch torchvision torchaudio -y
!pip install torch==1.11.0 torchvision==0.12.0


Found existing installation: torch 1.11.0
Uninstalling torch-1.11.0:
  Successfully uninstalled torch-1.11.0
Found existing installation: torchvision 0.12.0
Uninstalling torchvision-0.12.0:
  Successfully uninstalled torchvision-0.12.0
  Using cached torch-1.11.0-cp310-cp310-manylinux1_x86_64.whl (750.6 MB)
  Using cached torchvision-0.12.0-cp310-cp310-manylinux1_x86_64.whl (21.0 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.7.1 requires torch>=2, but you have torch 1.11.0 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 1.11.0 which is incompatible.


In [ ]:
import torch
print(torch.__version__)


1.11.0+cu102


In [ ]:
import os
import torch.multiprocessing
import multiprocessing

if multiprocessing.get_start_method(allow_none=True) != 'spawn':
    torch.multiprocessing.set_start_method('spawn', force=True)

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/GANsNFTs/StyleGAN/data/gan/experiments"
DATA = "/content/drive/MyDrive/GANsNFTs/StyleGAN/data/gan/dataset/portraits"
SNAP = 10

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py "\
  f"--snap {SNAP} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 10,
  "network_snapshot_ticks": 10,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/content/drive/MyDrive/GANsNFTs/StyleGAN/data/gan/dataset/portraits",
    "use_labels": false,
    "max_size": 560,
    "xflip": false,
    "resolution": 64
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "synthesis_kwargs": {
      "channel_base": 16384,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "ch

## **StyleGAN3 Deneme**

In [ ]:
!git clone https://github.com/NVlabs/stylegan3.git

Cloning into 'stylegan3'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 212 (delta 0), reused 1 (delta 0), pack-reused 207
Receiving objects: 100% (212/212), 4.16 MiB | 8.48 MiB/s, done.
Resolving deltas: 100% (101/101), done.


In [ ]:
%cd /content/stylegan3

/content/stylegan3


In [ ]:
import torch
print(torch.__version__)


2.2.1+cu121


In [ ]:
!pip install ninja

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 6.5 MB/s eta 0:00:00


In [ ]:
!python dataset_tool.py --source=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/images --dest=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/dataset/portraits.zip --resolution='64x64'

100% 560/560 [00:02<00:00, 264.79it/s]


In [ ]:
!python /content/stylegan3/train.py --outdir=~/training-runs --cfg=stylegan3-t --data=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/dataset/portraits.zip \
    --gpus=1 --batch=32 --batch-gpu=16 --gamma=6.6 --mirror=1 --kimg=500 --snap=50


Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan3.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "channel_base": 32768,
    "channel_max": 512,
    "magnitude_ema_beta": 0.9988915792636801
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
    "channel_max": 512
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 6.6
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "prefetch_fact

In [ ]:
!python /content/stylegan3/train.py --outdir=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/experiments/training-runs --cfg=stylegan3-t --data=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/dataset/portraits.zip \
    --gpus=1 --batch=32 --batch-gpu=16 --metrics=none --gamma=6.6 --mirror=1 --kimg=500 --snap=50


Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan3.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "channel_base": 32768,
    "channel_max": 512,
    "magnitude_ema_beta": 0.9988915792636801
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
    "channel_max": 512
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 6.6
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "prefetch_fact

In [ ]:
!python /content/stylegan3/train.py --outdir=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/experiments/training-runs \
    --cfg=stylegan3-t --data=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/dataset/portraits.zip \
    --gpus=1 --batch=32 --batch-gpu=16 --metrics=none --gamma=6.6 --mirror=1 --kimg=96 --snap=10 \
    --resume=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/experiments/training-runs/00000-stylegan3-t-portraits-gpus1-batch32-gamma6.6/network-snapshot-000500.pkl


Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan3.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "channel_base": 32768,
    "channel_max": 512,
    "magnitude_ema_beta": 0.9988915792636801
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
    "channel_max": 512
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 6.6,
    "blur_init_sigma": 0
  },
  "data_loader_kwargs": {
    "pin_memory"

In [ ]:
!python /content/stylegan3/gen_images.py --outdir=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/experiments/out --trunc=1 --seeds=84,264,296,850 \
    --network=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/experiments/portraits_256x256/training-runs/00000-stylegan3-t-portraits_256x256-gpus1-batch32-gamma6.6/network-snapshot-000040.pkl

python3: can't open file '/content/stylegan3/gen_images.py': [Errno 2] No such file or directory


## **Portraits 256x256**

###Environment

In [ ]:
!git clone https://github.com/NVlabs/stylegan3.git

Cloning into 'stylegan3'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 212 (delta 0), reused 1 (delta 0), pack-reused 207
Receiving objects: 100% (212/212), 4.16 MiB | 32.06 MiB/s, done.
Resolving deltas: 100% (101/101), done.


In [ ]:
!pip install ninja

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 6.7 MB/s eta 0:00:00


In [ ]:
%cd /content/drive/MyDrive/GANsNFTs/StyleGAN/gan/images/portraits/r256x256

/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/images/portraits/r256x256


In [ ]:
!python /content/stylegan3/dataset_tool.py --source=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/images/portraits/r256x256 --dest=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/dataset/portraits_256x256.zip --resolution='256x256'

100% 14965/14965 [04:26<00:00, 56.10it/s]


In [ ]:
!python /content/stylegan3/train.py --outdir=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/experiments/portraits_256x256/training-runs --cfg=stylegan3-t --data=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/dataset/portraits_256x256.zip \
    --gpus=1 --batch=32 --batch-gpu=16 --metrics=fid50k_full --gamma=6.6 --mirror=1 --kimg=500 --snap=10


Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan3.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "channel_base": 32768,
    "channel_max": 512,
    "magnitude_ema_beta": 0.9988915792636801
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
    "channel_max": 512
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 6.6
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "prefetch_fact

In [ ]:
!python /content/stylegan3/train.py --outdir=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/experiments/portraits_256x256/training-runs \
    --cfg=stylegan3-t --data=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/dataset/portraits_256x256.zip \
    --gpus=1 --batch=32 --batch-gpu=16 --metrics=fid50k_full --gamma=6.6 --mirror=1 --kimg=500 --snap=10 \
    --resume=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/experiments/portraits_256x256/training-runs/00000-stylegan3-t-portraits_256x256-gpus1-batch32-gamma6.6/network-snapshot-000040.pkl


Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan3.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "channel_base": 32768,
    "channel_max": 512,
    "magnitude_ema_beta": 0.9988915792636801
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
    "channel_max": 512
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 6.6,
    "blur_init_sigma": 0
  },
  "data_loader_kwargs": {
    "pin_memory"

In [ ]:
!python /content/stylegan3/gen_images.py --outdir=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/experiments/portraits_256x256/out/model02 --trunc=1 --seeds=84,264,296,850 \
    --network=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/experiments/portraits_256x256/training-runs/00007-stylegan3-t-portraits_256x256-gpus1-batch32-gamma6.6/network-snapshot-000000.pkl

Loading networks from "/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/experiments/portraits_256x256/training-runs/00007-stylegan3-t-portraits_256x256-gpus1-batch32-gamma6.6/network-snapshot-000000.pkl"...
Generating image for seed 84 (0/4) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.
Generating image for seed 264 (1/4) ...
Generating image for seed 296 (2/4) ...
Generating image for seed 850 (3/4) ...


In [ ]:
!python /content/stylegan3/train.py --outdir=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/experiments/portraits_256x256/training-runs \
    --cfg=stylegan3-t --data=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/dataset/portraits_256x256.zip \
    --gpus=1 --batch=32 --batch-gpu=16 --metrics=fid50k_full --gamma=6.6 --mirror=1 --kimg=300 --snap=10 \
    --resume=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/experiments/portraits_256x256/training-runs/00002-stylegan3-t-portraits_256x256-gpus1-batch32-gamma6.6/network-snapshot-000000.pkl


Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan3.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "channel_base": 32768,
    "channel_max": 512,
    "magnitude_ema_beta": 0.9988915792636801
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
    "channel_max": 512
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 6.6,
    "blur_init_sigma": 0
  },
  "data_loader_kwargs": {
    "pin_memory"

###Resume without metrics

In [ ]:
!python /content/stylegan3/train.py --outdir=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/experiments/portraits_256x256/training-runs \
    --cfg=stylegan3-t --data=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/dataset/portraits_256x256.zip \
    --gpus=1 --batch=32 --batch-gpu=16 --metrics=none --gamma=6.6 --mirror=1 --kimg=300 --snap=10 \
    --resume=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/experiments/portraits_256x256/training-runs/00003-stylegan3-t-portraits_256x256-gpus1-batch32-gamma6.6/network-snapshot-000000.pkl


Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan3.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "channel_base": 32768,
    "channel_max": 512,
    "magnitude_ema_beta": 0.9988915792636801
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
    "channel_max": 512
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 6.6,
    "blur_init_sigma": 0
  },
  "data_loader_kwargs": {
    "pin_memory"